In [1]:
import numpy as np

from typing import Type, Union

from aeon.classification.distance_based import ProximityTree
from sklearn.utils import check_random_state

from aeon.classification.base import BaseClassifier

In [2]:
class ProximityForest(BaseClassifier):
    """Proximity Forest Classifier.
    
    The Proximity Forest is an ensemble of Proximity Trees."""

    def __init__(
        self,
        n_trees=10,
        n_splitters: int = 5,
        max_depth: int = None,
        min_samples_split: int = 2,
        random_state: Union[int, Type[np.random.RandomState], None] = None,
        n_jobs: int = 1
    ):
        self.n_trees = n_trees
        self.n_splitters = n_splitters
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.n_jobs = n_jobs
        super().__init__()

    def _fit(self, X, y):
        # Check dimension of X
        if X.ndim == 3:
            if X.shape[1] == 1:
                X = np.squeeze(X, axis=1)
            else:
                raise ValueError("X should be univariate.")

        self.classes_ = list(np.unique(y))
        self.trees_ = []
        for i in range(self.n_trees):
            clf = ProximityTree(
                n_splitters = self.n_splitters,
                max_depth = self.max_depth,
                min_samples_split = self.min_samples_split,
                random_state = self.random_state,
                n_jobs = self.n_jobs)
            clf.fit(X,y)
            self.trees_.append(clf)
            
        self._is_fitted = True

    def _predict_proba(self, X):
        # Check dimension of X
        if X.ndim == 3:
            if X.shape[1] == 1:
                X = np.squeeze(X, axis=1)
            else:
                raise ValueError("X should be univariate.")

        output_probas = []
        for i in range(self.n_trees):
            proba = self.trees_[i].predict_proba(X)
            output_probas.append(proba)

        output_probas = np.sum(output_probas, axis=0)
        output_probas = np.divide(output_probas, self.n_trees)
        return output_probas
        
    def _predict(self, X):
        probas = self._predict_proba(X)
        idx = np.argmax(probas, axis=1)
        preds = np.asarray([self.classes_[x] for x in idx])
        return preds

In [3]:
from aeon.datasets import load_arrow_head
X, y = load_arrow_head(split='TEST', return_type = 'numpy3d')
X.shape

(175, 1, 251)

In [4]:
X_test, y_test = load_arrow_head(split='TRAIN', return_type='numpy3d')
X_test.shape

(36, 1, 251)

In [5]:
import time

In [6]:
start = time.perf_counter()
clf = ProximityForest()
end = time.perf_counter()
print("Elapsed = {}s".format((end - start)))

Elapsed = 0.00018369988538324833s


In [7]:
start = time.perf_counter()
clf.fit(X,y)
end = time.perf_counter()
print("Elapsed = {}s".format((end - start)))

Elapsed = 50.049295200034976s


In [8]:
start = time.perf_counter()
y_pred = clf.predict(X_test)
end = time.perf_counter()
print("Elapsed = {}s".format((end - start)))

Elapsed = 1.6872906000353396s


In [9]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
score

0.9444444444444444

In [10]:
y_ = clf.predict(X)
accuracy_score(y_,y)

1.0

In [11]:
clf.predict_proba(X_test)

array([[1.  , 0.  , 0.  ],
       [0.1 , 0.8 , 0.1 ],
       [0.  , 0.4 , 0.6 ],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.55, 0.25, 0.2 ],
       [0.65, 0.35, 0.  ],
       [0.25, 0.65, 0.1 ],
       [0.  , 0.1 , 0.9 ],
       [1.  , 0.  , 0.  ],
       [0.1 , 0.5 , 0.4 ],
       [0.  , 0.1 , 0.9 ],
       [0.85, 0.15, 0.  ],
       [0.  , 1.  , 0.  ],
       [0.25, 0.2 , 0.55],
       [0.95, 0.05, 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ],
       [0.15, 0.85, 0.  ],
       [0.45, 0.  , 0.55],
       [0.7 , 0.3 , 0.  ],
       [0.25, 0.75, 0.  ],
       [0.3 , 0.  , 0.7 ],
       [0.9 , 0.1 , 0.  ],
       [0.05, 0.95, 0.  ],
       [0.8 , 0.1 , 0.1 ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.9 , 0.1 ],
       [0.  , 0.  , 1.  ],
       [0.95, 0.05, 0.  ],
       [0.05, 0.85, 0.1 ],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ]])

In [12]:
from aeon.classification.distance_based import ProximityTree

clf1 = ProximityTree()
start = time.perf_counter()
clf1.fit(X,y)
end = time.perf_counter()
print("Elapsed = {}s".format((end - start)))

Elapsed = 5.014899000059813s


In [13]:
y_pred1 = clf1.predict(X_test)
score1 = accuracy_score(y_pred1, y_test)
score1

0.6944444444444444

In [ ]:
from aeon.classification.distance_based import ElasticEnsemble

clf1 = ElasticEnsemble(distance_measures=['dtw','lcss','twe','ddtw'])
start = time.perf_counter()
clf1.fit(X,y)
end = time.perf_counter()
print("Elapsed = {}s".format((end - start)))